In [2]:
import os, sys, time

import requests
import shutil

# https://gitlab.com/dlr-ve-esy/esy-osmfilter/-/tree/master/
from esy.osmfilter import  osm_colors          as CC
from esy.osmfilter import run_filter, Node,Way,Relation 
from esy.osmfilter import export_geojson
from contextlib import contextmanager

# import logging
# logging.basicConfig()
# logger=logging.getLogger(__name__)
# logger.setLevel(logging.INFO)
# logger.setLevel(logging.WARNING)


In [3]:
country_name = 'nigeria' # TODO: Create dictionary with country_code : country_name pairs
# country_name = 'cape-verde'
geofabrik_filename=f'{country_name}-latest.osm.pbf' # Filename for geofabrik 
geofabrik_url = f'https://download.geofabrik.de/africa/{geofabrik_filename}' # https://download.geofabrik.de/africa/nigeria-latest.osm.pbf

local_file= os.path.join("data","osm",geofabrik_filename)
if not os.path.exists(local_file):
    print(f"{geofabrik_filename} does not exist, downloading to {local_file}")
    os.makedirs(os.path.dirname(local_file), exist_ok=True) # create data/osm directory
    with requests.get(geofabrik_url, stream=True) as r:
        with open(local_file, 'wb') as f:
            shutil.copyfileobj(r.raw, f)

# Input filepath
PBF_inputfile = os.path.join(os.getcwd(),local_file)

# json file for the Data dictionary 
# json file for the Elements dictionary is automatically written to 'data/osm/Elements'+filename)
JSON_outputfile = os.path.join(os.getcwd(),'data','osm',country_name+'_substations.json')








In [4]:
print(PBF_inputfile, JSON_outputfile)

/home/matin/pypsa-africa/data_exploration/data/osm/nigeria-latest.osm.pbf /home/matin/pypsa-africa/data_exploration/data/osm/nigeria_substations.json


In [5]:
#see https://dlr-ve-esy.gitlab.io/esy-osmfilter/filter.html for filter structures

prefilter = {Node: {"power":["substation"]}, Way: {"power":["substation"]}, Relation: {"power":["substation"]}}
blackfilter = [("pipeline","substation"),]
whitefilter =[[("power","substation"),],]
    
elementname='NG_substations'

[Data,Elements]=run_filter(elementname,PBF_inputfile, JSON_outputfile,prefilter,whitefilter,blackfilter, NewPreFilterData=True, CreateElements=True, LoadElements=True,verbose=True,multiprocess=True)




INFO:esy.osmfilter.pre_filter:PreFilter OSM GAS DATA
INFO:esy.osmfilter.pre_filter:InputFile     : /home/matin/pypsa-africa/data_exploration/data/osm/nigeria-latest.osm.pbf
INFO:esy.osmfilter.pre_filter:Size          : 303051          kbyte
INFO:esy.osmfilter.pre_filter:Estimated Time: 43.29           s
INFO:esy.osmfilter.pre_filter:=============================
INFO:esy.osmfilter.pre_filter:0.5
INFO:esy.osmfilter.pre_filter:1
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_fi

In [9]:
print(Elements)

# export the elements to a geojson-file
# export_geojson(Elements[elementname]['Way'],Data,filename='test.geojson',jsontype='Line')
# convert extract node elements
# export_geojson(Elements[elementname]['Node'],Data,filename='test.geojson',jsontype='Point')

{'NG_substations': {'Node': {}, 'Way': {}, 'Relation': {}}}
